**テキスト要約を学習(Fine Tuning)するための学習データを収集**

In [ ]:
!git clone https://github.com/KodairaTomonori/ThreeLineSummaryDataset

In [ ]:
import os
from pprint import pprint
import random
import time

from bs4 import BeautifulSoup
from bs4.element import NavigableString
import pandas as pd
from urllib.request import urlopen

In [ ]:
# 本文と要約をホームページから取得するための関数
def get_content(id):
    interval_time = random.uniform(2, 3)
    time.sleep(interval_time)
    URL = "https://news.livedoor.com/article/detail/" + id+ "/"
    print(URL)
    try:
        with urlopen(URL) as res:
            # 本文を取得する
            output1 = ""
            html = res.read().decode("euc_jp", "ignore")
            soup = BeautifulSoup(html, "html.parser")
            lineList = soup.select(".articleBody p")
            for line in lineList:
                if len(line.contents) > 0 and type(line.contents[0]) == NavigableString:
                    output1 += line.contents[0].strip()
            if output1 == "":  # 本文がホームページに無かった場合は処理を抜ける
                return
            output1 += "\n"
            # 要約を取得する
            output0 = ""
            summaryList = soup.select(".summaryList li")
            for summary in summaryList:
                output0 += summary.contents[0].strip() + "。"
            if output0 == "":  # 要約がホームページに無かった場合は処理を抜ける
                return
            # csvファイルに書き込む
            with open("./output_summary_**.csv", mode="a") as f:
                f.writelines(output0 + "\t" + output1)
    except Exception:
        print("Exception")

In [ ]:
# 本文と要約をホームページから取得する
idList = []
with open("ThreeLineSummaryDataset/data/train.csv", mode="r") as f:
    lines = f.readlines()
    for line in lines:
        id = line.strip().split(",")[3].split(".")[0]
        idList.append(id)
for i in range(119000, 120000):  # 3秒毎に本文と要約を1つ取得する
    print("{a}つ目".format(a=i+1))
    get_content(idList[i])

In [ ]:
# 複数のcsvファイルの形で収集した本文と要約を1つのcsvファイルに纏める
df_all = pd.read_csv("./output_summary_all.csv")
with open("./output_summary_12.csv", encoding="shift-jis") as f:
    for line in f.readlines():
        strs = line.split("\t")
        if len(strs) == 2:
            df_temp = pd.DataFrame(data={"summary": [strs[0]],
                                         "text": [strs[1]]})
            df_all = pd.concat(objs=[df_all, df_temp],
                               ignore_index=True)
df_all.to_csv("./output_summary_all.csv",
              index=False)

In [ ]:
pd.read_csv("./output_summary_all.csv")

,summary,text
0,岡むら屋から、期間限定の新メニュー「じゃが肉めし」が登場する。男爵いもなどは味噌ベースで煮こ...,新橋と秋葉原に店を構える「岡むら屋」。味噌ベースの独自の味つけで牛バラ肉を煮込んだ具材がたっ...
1,東京駅周辺の安くて美味しい「蕎麦ランチ」の名店を紹介している。「越後そば 東京店」では、ミニ...,名店がひしめく「丸の内・日本橋」エリアで＜うまい蕎麦ランチ＞が食べられるお店を厳選してご紹介...
2,8日から「サンクトガーレン」は、「チョコビール」4種を販売する。ダークな茶色、ほろ苦いビター...,地ビールメーカー「サンクトガーレン」は、「チョコビール」4種を2016年1月8日から全国の取...
3,1年以内に登場した、東京みやげの新定番を紹介している。カファレルの「東京ジャンドゥーヤチョコ...,何かと人に会う機会が多いこの時期。そんな時、頭を悩ませるのが手みやげ選びだ。毎年似たようなも...
4,新しいポテトチップスの味として「パクチー味」が話題になっている。フリーズドライパクチーとパク...,新しいポテトチップスの味として今、話題になっているのが「それだけではありません。さらに、普通...
...,...,...
94574,ロッテは23日から、「雪見だいふく クッキー＆クリーム」を発売する。雪見だいふく初のクッキー...,ロッテアイスは9月23日から、シリーズ初のクッキークランチ入り「「雪見だいふく クッキー＆ク...
94575,match.comが独身男女289人に対し、地元志向についてアンケートした。将来的に地元で暮...,一昨年の大震災以降、「家族の絆」が強く意識されるようになり、結婚相手に同郷の人を選ぶ傾向が強...
94576,ITが苦手な人への嫌がらせ「テクハラ」が問題となっている。PCの操作に疎い人に「こんな簡単な...,仕事に必要なパソコン業務が増える中、それに対応できない人への「テクハラ」が、問題視され始めて...
94577,女性400名に、車の運転で苦手な動作をアンケートした。1位は44.8％が答えた縦列駐車で、2...,久しぶりのディズニーリゾート。早めに出発したのに、駐車場はもう大混雑! 後続車のドライバーが...


In [ ]:
# 収集した学習データをtrainとvalidationとtestに分けて、それぞれをcsvファイルにする
df = pd.read_csv("./output_summary_all.csv")
df = df.sample(frac=1)  # DataFrame内のデータをシャッフル
num = len(df)
df.iloc[:int(num*0.85), :].to_csv("./train.csv",  # 計算結果に小数点があってもintで整数にする
                                  index=False)
df.iloc[int(num*0.85):int(num*0.925), :].to_csv("./valid.csv",
                                                index=False)
df.iloc[int(num*0.925):, :].to_csv("./test.csv",
                                   index=False)